In [1]:
# 从表格数据构建节点列表
data = [
    (1, "AI", 0),
    (2, "Model", 1),
    (3, "CorssEncoder", 5),
    (4, "BiEncoder", 5),
    (5, "RankModel", 2),
    (6, "CoT", 7),
    (7, "Reason", 1),
    (8, "ReACT", 7),
    (9, "RAG", 1),
    (10, "Agent", 1),
    (11, "EmbeddingModel", 2),
    (12, "RetroMAE", 11)
]

# 创建DataFrame
import pandas as pd
df = pd.DataFrame(data, columns=['id', 'name', 'parent_id'])

In [2]:
def find_all_children(df):
    """计算每个节点的所有孩子节点（直接和间接）"""
    # 构建父子关系映射
    parent_to_children = {}
    id_to_name = dict(zip(df['id'], df['name']))
    
    # 初始化每个节点的直接孩子
    for _, row in df.iterrows():
        parent_id = row['parent_id']
        child_id = row['id']
        
        if parent_id not in parent_to_children:
            parent_to_children[parent_id] = []
        parent_to_children[parent_id].append(child_id)
    
    # 递归查找所有孩子节点
    def find_descendants(node_id):
        """递归查找节点的所有后代节点"""
        if node_id not in parent_to_children:
            return []
        
        descendants = []
        for child_id in parent_to_children[node_id]:
            descendants.append(child_id)
            descendants.extend(find_descendants(child_id))
        
        return descendants
    
    # 计算每个节点的所有孩子节点
    all_children = {}
    for node_id in df['id']:
        all_children[node_id] = find_descendants(node_id)
    
    return all_children, id_to_name

# 计算所有孩子节点
all_children, id_to_name = find_all_children(df)

In [3]:
all_children

{1: [2, 5, 3, 4, 11, 12, 7, 6, 8, 9, 10],
 2: [5, 3, 4, 11, 12],
 3: [],
 4: [],
 5: [3, 4],
 6: [],
 7: [6, 8],
 8: [],
 9: [],
 10: [],
 11: [12],
 12: []}

TypeError: list indices must be integers or slices, not str

'科技馆有哪些类型的活动？ 苏州科技馆主要包括以下活动种类：依托展厅展品开发的教育课程与个性化讲解服务；在专属空间开展的科学教室系统培训；结合场馆与外部资源的研学旅行；可灵活使用的实验探究材料箱；通过线上平台传播的数字资源；覆盖权威竞赛与趣味活动的科技赛事以及融合讲座、表演、市集等形式的主题特色活动。\n\n苏州科技馆有别于其它科技馆的特色是什么？ 苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着力于打造与传统科技馆具有明显差异化的、特色鲜明的科技馆。\n\n介绍一下苏州科技馆 苏州科技馆位于苏州高新区狮山文化广场，总建筑面积约6.3万平方米，可布展面积约4万平方米。载体以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，包括“微中有机”、“无微不至”、“探微历程”、“微力无穷”、“察微知著”等10个常设展厅，以及3个临展厅、3个特效影院、7个展教实验空间等。苏州科技馆通过数字化沉浸式展陈体验、信息化场馆和智慧服务平台建设、受众与产品大数据分析等手段，着力于打造与传统科技馆具有明显差异化的、特色鲜明的新型科技馆。苏州科技馆运用数字化科技手段，围绕展教培训、科普活动、文创开发等业务，提升产品和服务品质，转化产业价值，力争建设成为面向长三角乃至全国青少年群体的具有数字化特色的科普阵地。\n\n科技馆有哪些类型的培训？ 1、素质拓展型：通过展厅活动、科学课程和研学服务，培养科学思维与动手能力，提升综合科学素养。2、学业辅助型：依托科学教室系列课程及实验材料箱，提供课后培训、巩固和拓展科学知识，助力学业水平提升。3、竞赛挑战型：围绕白名单等权威赛事开展培训，帮助学生获取竞赛成果，增强升学竞争力。\n\n是否可以使用无人机或使用无人机拍摄？ 禁止使用无人机\n\n苏州科技馆为什么以微生物为主题？ 苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着力于打造与传统科技馆具有明

{'data_str': '科技馆有哪些类型的活动？ 苏州科技馆主要包括以下活动种类：依托展厅展品开发的教育课程与个性化讲解服务；在专属空间开展的科学教室系统培训；结合场馆与外部资源的研学旅行；可灵活使用的实验探究材料箱；通过线上平台传播的数字资源；覆盖权威竞赛与趣味活动的科技赛事以及融合讲座、表演、市集等形式的主题特色活动。\n\n苏州科技馆有别于其它科技馆的特色是什么？ 苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着力于打造与传统科技馆具有明显差异化的、特色鲜明的科技馆。\n\n介绍一下苏州科技馆 苏州科技馆位于苏州高新区狮山文化广场，总建筑面积约6.3万平方米，可布展面积约4万平方米。载体以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，包括“微中有机”、“无微不至”、“探微历程”、“微力无穷”、“察微知著”等10个常设展厅，以及3个临展厅、3个特效影院、7个展教实验空间等。苏州科技馆通过数字化沉浸式展陈体验、信息化场馆和智慧服务平台建设、受众与产品大数据分析等手段，着力于打造与传统科技馆具有明显差异化的、特色鲜明的新型科技馆。苏州科技馆运用数字化科技手段，围绕展教培训、科普活动、文创开发等业务，提升产品和服务品质，转化产业价值，力争建设成为面向长三角乃至全国青少年群体的具有数字化特色的科普阵地。\n\n科技馆有哪些类型的培训？ 1、素质拓展型：通过展厅活动、科学课程和研学服务，培养科学思维与动手能力，提升综合科学素养。2、学业辅助型：依托科学教室系列课程及实验材料箱，提供课后培训、巩固和拓展科学知识，助力学业水平提升。3、竞赛挑战型：围绕白名单等权威赛事开展培训，帮助学生获取竞赛成果，增强升学竞争力。\n\n是否可以使用无人机或使用无人机拍摄？ 禁止使用无人机\n\n苏州科技馆为什么以微生物为主题？ 苏州科技馆以“揭示微观世界奥秘、激发科技兴趣、启迪人生哲思”为主题，以“认识你自己（认识生命的奥秘）”、“认知人类赖以生存的微观生态系统”、“展示人类探索微观世界的历史”、“揭示面向微观世界的最新科技成果应用场景”和“仰望星空、放飞梦想”为策展思路，着

<re.Match object; span=(23, 44), match='问题:介绍一下趣波，介绍一下数字人\n回复:'>

<re.Match object; span=(41, 458), match='回复:苏州科技馆IP档案:\n中文名:趣波\n英文名:QuBoo(Qu(量子Quantum/探索Q>